In [171]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from functools import reduce

In [172]:
%connect_info
%qtconsole


{
  "shell_port": 57966,
  "iopub_port": 57967,
  "stdin_port": 57968,
  "control_port": 57970,
  "hb_port": 57969,
  "ip": "127.0.0.1",
  "key": "1a90cb15-87bd8bf4a1fa2295d26ba593",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-9e42a008-5430-4096-8c2c-ff77f1e09a61.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [173]:
pwd

'/Users/cgu3/Library/CloudStorage/OneDrive-InsideMDAnderson/untitled folder/pytorch_geometric/graphgym/analysis'

In [174]:
gs1 = pd.read_csv('../data/yeast/raw/reference/goldstandard1.txt', header=None, index_col=False, names=['protein'])
gs2 = pd.read_csv('../data/yeast/raw/reference/goldstandard2.txt', header=None, index_col=False, names=['protein'])
gs3 = pd.read_csv('../data/yeast/raw/reference/goldstandard3.txt', header=None, index_col=False, names=['protein'])
gs4 = pd.read_csv('../data/yeast/raw/reference/goldstandard4.txt', header=None, index_col=False, names=['protein'])
all = pd.read_csv('../data/yeast/raw/reference/yeast_all.lst.txt', header=None, index_col=False, names=['protein'])

In [175]:
gs1['detected'] = 1
gs2['detected'] = 1
gs3['detected'] = 1
gs4['detected'] = 1

In [176]:
gs1

,protein,detected
0,Q0250,1
1,YAL003W,1
2,YAL005C,1
3,YAL012W,1
4,YAL015C,1
...,...,...
1998,YPR184W,1
1999,YPR187W,1
2000,YPR189W,1
2001,YPR190C,1


In [177]:
data_frames = [gs1, gs2, gs3, gs4]


In [178]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['protein'],
                                            how='outer'), data_frames).fillna(0)

/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_57272/1543971361.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'detected_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['protein'],


In [179]:
df_merged.columns = ['protein', 'gs1', 'gs2', 'gs3', 'gs4']

In [180]:

df_merged = df_merged.set_index('protein')

In [181]:
positive_protein = df_merged[df_merged.sum(axis=1) >= 2].index.to_series()

In [182]:
df_merged[df_merged.sum(axis=1) == 0]

,gs1,gs2,gs3,gs4
protein,,,,


In [183]:
protein_with_evidence = df_merged.index.to_series()

In [184]:
all_protein = all.iloc[:, 0]

In [185]:
negative_protein = all_protein[~all_protein.isin(protein_with_evidence)]

In [206]:
protein_MS = pd.read_csv('../data/yeast/raw/protein/yeast_rich_medium.csv')
protein_MS.dtypes
protein_MS.head()
protein_MS['protein'] = protein_MS['protein'].str.strip()

entry no.                         object
protein                           object
protein probability              float64
protein description               object
protein length                     int64
percent coverage                 float64
tot indep spectra                  int64
percent share of spectrum ids    float64
peptides                          object
dtype: object

,entry no.,protein,protein probability,protein description,protein length,percent coverage,tot indep spectra,percent share of spectrum ids,peptides
0,1,YAL003W,1.0,"EFB1 SGDID:S000000003,'Translation elongation ...",0,42.7,312,0.425,AFQSAYPEFSR+AIEMEGLTWGAHQFIPIGFGIK+IETLKQLNASL...
1,2,YAL005C,1.0,"SSA1 SGDID:S000000004,'ATPase involved in prot...",0,54.5,838,0.797,AEETISWLDSNTTASKEEFDDKLK+ARFEELCADLFR+ATAGDTHL...
2,3,YAL012W,1.0,"CYS3 SGDID:S000000010,'Cystathionine gamma-lya...",0,36.5,235,0.321,AIHAGEHVDVHGSVIEPISLSTTFK+DALGGGMISFR+EASGVFDD...
3,4,YAL016W,1.0,"TPD3 SGDID:S000000014,'Regulatory subunit A of...",0,13.9,17,0.021,DSDSNESLYPLALLMDELK+FLNDPSIILNK+ILPAVQNLSMDESE...
4,5,YAL021C,1.0,"CCR4 SGDID:S000000019,'Component of the CCR4-N...",0,5.7,8,0.007,DAMDFSGAWMKHKK+FIGFPNDKFPSDHIPLLAR+LSQYSIDEDDD...


In [210]:
protein_MS['y'] = 3
protein_MS.loc[protein_MS['protein'].isin(list(positive_protein)), 'y'] = 1
protein_MS.loc[protein_MS['protein'].isin(list(negative_protein)), 'y'] = 0

In [211]:
protein_MS['y']

0       1
1       1
2       1
3       1
4       1
       ..
3848    0
3849    0
3850    0
3851    1
3852    0
Name: y, Length: 3853, dtype: int64

In [212]:
protein_MS['y'].value_counts()

0    1463
1    1413
3     977
Name: y, dtype: int64